<a href="https://colab.research.google.com/github/miiiingi/dacon_sentiment/blob/main/dacon_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/gdrive/')
my_folder = '/gdrive/MyDrive/ColabNotebooks/dacon_senti/dataset/dataset'

Mounted at /gdrive/


In [231]:
import pandas as pd
import numpy as np
!pip install konlpy
from konlpy.tag import Mecab
from konlpy.tag import Okt
from tqdm import tqdm
from torchtext.legacy import data, datasets
import torch
import random
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

Torchtext - Field 정의

In [248]:
okt = Okt()
ID = data.Field(use_vocab=False, sequential=False, batch_first= True)
TEXT = data.Field(use_vocab = True, tokenize=okt.morphs, sequential= True, batch_first = True)
LABEL = data.Field(use_vocab= False, sequential= False, is_target=True, batch_first= True)
fields = [('id', ID), ('document',TEXT), ('label',LABEL)]

Torchtext - 데이터셋 불러와서 정의

In [249]:
train_data, test_data = data.TabularDataset.splits(
                            path = f'{my_folder}/',
                            train = 'train.csv',
                            test = 'test.csv',
                            format = 'csv',
                            fields = fields,
                            skip_header = True,
)
train_data, valid_data = train_data.split(random_state=random.seed(722), split_ratio = 0.5)

Torchtext - 단어 집합 만들기

In [250]:
TEXT.build_vocab(train_data, min_freq=5) # 단어 집합 생성
LABEL.build_vocab(train_data)

Torchtext - 배치화 시키기

In [251]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (train_data, valid_data, test_data), batch_size=32, sort_key = lambda x: len(x.document), sort_within_batch = True,
        shuffle=True, repeat=False)

GRU Model 정의

In [264]:
from torch import nn as nn 
import torch.nn.functional as F
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers * 2,
                          batch_first=True,
                          bidirectional = True)
        self.out = nn.Linear(self.hidden_dim * 2, n_classes)

    def forward(self, x):
        x = self.embed(x)
        # h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [277]:
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model = GRU(1, 256, len(TEXT.vocab), 128, 2, 0.5).to(DEVICE)
# model = GRU(1, 256, len(TEXT.vocab), 128, 2, 0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [279]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.document.to(DEVICE), batch.label.to(DEVICE)
        optimizer.zero_grad()
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [280]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.document.to(DEVICE), batch.label.to(DEVICE)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [ ]:
best_val_loss = None
for e in tqdm(range(300)):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e+1, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), f'{my_folder}/snapshot/sentiment.pt')
        best_val_loss = val_loss

  0%|          | 0/300 [00:00<?, ?it/s]